In [ ]:
print(__doc__)
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

import scipy.io as sio

mat_contents = sio.loadmat('umist_cropped.mat')
facedat = mat_contents['facedat']

people = facedat[0]
people1 = facedat[0][0]
people2 = facedat[0][1]
people1_face1 = facedat[0][0][:,:,0]
people1_face2 = facedat[0][0][:,:,1]
people2_face2 = facedat[0][1][:,:,1]
n_samples = 20
n_neighbors = 10
n_components = 2

seed = np.random.RandomState(seed=3)
# Center the data
X_true = people1_face1
X_true -= X_true.mean()

similarities = euclidean_distances(X_true)

# Add noise to the similarities
# noise = np.random.rand(n_samples, n_samples)
# noise = noise + noise.T
# noise[np.arange(noise.shape[0]), np.arange(noise.shape[0])] = 0
# similarities += noise

mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds.fit(similarities).embedding_

isomap = manifold.Isomap(n_neighbors, n_components)
ipos = isomap.fit(similarities).embedding_

lle = manifold.LocallyLinearEmbedding(n_neighbors, n_components,
                                        eigen_solver='auto',
                                        method='standard')
lpos = lle.fit(similarities).embedding_

ltsa = manifold.LocallyLinearEmbedding(n_neighbors, n_components,
                                       eigen_solver='auto',
                                       method = 'ltsa')
sapos = ltsa.fit(similarities).embedding_

le = manifold.SpectralEmbedding(n_components = n_components,
                                n_neighbors = n_neighbors)
lepos = le.fit(similarities).embedding_

mvu = manifold.TSNE(n_components=n_components, init = 'pca', random_state=0)
mpos = mvu.fit(similarities).embedding_


# nmds = manifold.MDS(n_components=2, metric=False, max_iter=3000, eps=1e-12,
#                     dissimilarity="precomputed", random_state=seed, n_jobs=1,
#                     n_init=1)
# npos = nmds.fit_transform(similarities, init=pos)



# Rescale the data
pos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((pos ** 2).sum())
#npos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((npos ** 2).sum())
ipos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((ipos ** 2).sum())
lpos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((lpos ** 2).sum())
sapos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((sapos ** 2).sum())
lepos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((lepos ** 2).sum())
mpos *= np.sqrt((X_true ** 2).sum()) / np.sqrt((mpos ** 2).sum())

# Rotate the data
clf = PCA(n_components=2)
X_true = clf.fit_transform(X_true)

pos = clf.fit_transform(pos)

#npos = clf.fit_transform(npos)

ipos = clf.fit_transform(ipos)

lpos = clf.fit_transform(lpos)

sapos = clf.fit_transform(sapos)

lepos = clf.fit_transform(lepos)

mpos = clf.fit_transform(mpos)

fig = plt.figure(1)
ax = plt.axes([0., 0., 1., 1.])

plt.scatter(X_true[:, 0], X_true[:, 1], c='r', s=20)
plt.scatter(pos[:, 0], pos[:, 1], s=20, c='g')
#plt.scatter(npos[:, 0], npos[:, 1], s=20, c='b')
plt.scatter(ipos[:, 0], ipos[:, 1], s=20, c='b')
plt.scatter(lpos[:, 0], lpos[:, 1], s=20, c='y')
plt.scatter(sapos[:, 0], sapos[:, 1], s=20, c='c')
plt.scatter(lepos[:, 0], lepos[:, 1], s=20, c='m')
plt.scatter(mpos[:, 0], mpos[:, 1], s=20, c='k')

plt.legend(('True position', 'MDS', 'ISOMAP', 'LLE', 'LTSA', 'LE (SpectralEmbedding)','MVU'), loc='best')

similarities = similarities.max() / similarities * 100
similarities[np.isinf(similarities)] = 0

# Plot the edges
start_idx, end_idx = np.where(pos)
#a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[X_true[i, :], X_true[j, :]]
            for i in range(len(pos)) for j in range(len(pos))]
values = np.abs(similarities)
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, values.max()))
lc.set_array(similarities.flatten())
lc.set_linewidths(0.5 * np.ones(len(segments)))
ax.add_collection(lc)

plt.show()


Automatically created module for IPython interactive environment


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:29: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:112: RuntimeWarning: divide by zero encountered in divide
